In [1]:
import os

In [2]:
os.chdir("/Users/richcriticism/Documents/Documents/Bhavya_code/machine learning/CorrectPose2")

In [3]:
%pwd

'/Users/richcriticism/Documents/Documents/Bhavya_code/machine learning/CorrectPose2'

In [4]:
import tensorflow as tf
import keras


In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    all_params: dict
    params_batch_size: int
    eval_data_path: Path
    eval_data_target: Path


In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/base_model.h5",
            all_params=self.params,
            params_batch_size=self.params.BATCH_SIZE,
            eval_data_path = "artifacts/data_extraction/extracted_data/eval_Data_X.csv",
            eval_data_target = "artifacts/data_extraction/extracted_data/eval_Data_Y.csv"
        )
        return eval_config


In [8]:
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import numpy as np

In [9]:
class Evaluation:
    def __init__(self, config : EvaluationConfig):
        self.config = config


    def extract_eval_data(self):
        X = pd.read_csv(self.config.eval_data_path)
        Y = pd.read_csv(self.config.eval_data_target)


        return X,Y

    @staticmethod
    def _eval_data_generator(X,Y, config):

        params_batch_size = config.params_batch_size

        num_samples = len(X)
        while True:

            indices = np.random.permutation(num_samples)
            for i in range(0, num_samples, params_batch_size):

                if i+params_batch_size > num_samples:
                    break
                batch_indices = indices[i:i+params_batch_size]
                batch_x = X.iloc[batch_indices]
                batch_y = Y.iloc[batch_indices]
                yield np.array(batch_x), np.array(batch_y)
        
    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01), loss = tf.keras.losses.binary_crossentropy, metrics=['accuracy'])
        X,Y = self.extract_eval_data()
        eval_data_generator = self._eval_data_generator(X,Y, self.config)
        self.score = self.model.evaluate(eval_data_generator)

    def save_score(self):
        scores = {"loss" : self.score[0], "accuracy": self.score[1]}
        save_json(path = Path("scores.json"), data = scores)

In [10]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2024-04-09 20:01:04,314: INFO : common : yaml file : config/config.yaml loaded successfully]
[2024-04-09 20:01:04,316: INFO : common : yaml file : params.yaml loaded successfully]
[2024-04-09 20:01:04,316: INFO : common : creating directory at path: artifacts]
[2024-04-09 20:01:04,509: WARNING : saving_utils : Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
